## Task 1

**Importing Libraries**

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
import tensorflow
from tensorflow.keras.models import load_model

**Defining model for Age and Gender Detection**

In [4]:
age_gender_model = load_model("Age_Sex_Detection.keras")

**Loading Yolov3 weights and configuration**

In [23]:
#net = cv2.dnn.readNet('yolov3.weights','yolov3.cfg')
#layer_names = net.getLayerNames()
#output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

**Loading YOLOv5 model**

In [3]:
yolov5_model = torch.hub.load('ultralytics/yolov5','yolov5l')

Using cache found in C:\Users\hp/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-6-24 Python-3.12.4 torch-2.3.1+cpu CPU

Fusing layers... 
YOLOv5l summary: 367 layers, 46533693 parameters, 0 gradients, 109.0 GFLOPs
Adding AutoShape... 


In [4]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((48,48)),
    transforms.ToTensor(),
    transforms.Normalize([0.5],[0.5])
])

**Detecting People using YOLO**

In [5]:
def detect_people(image): # Returns vertices of the boxes of the faces
    results = yolov5_model(image)
    detections = results.xyxy[0].cpu().numpy() # Returns coordinates of each detection along with class and their confidences
    boxes = []
    for detection in detections:
        x1,y1,x2,y2,confidence,obj = detection
        if int(obj) == 0: # Since in COCO Dataset, 0 is for person
            x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
            boxes.append((x1,y1,x2-x1,y2-y1))    
    return boxes

In [8]:
boxes = detect_people(image)

**Detecting Shirt colour**

In [6]:
def detect_shirt_colour(image,box):
    x,y,w,h = box
    x,y,w,h = x,y+h//2,w,h//4
    person_img = image[y:y+h,x:x+w]
    hsv = cv2.cvtColor(person_img,cv2.COLOR_BGR2HSV)
    mask_white = cv2.inRange(hsv,(0,0,200),(180,30,255))
    mask_black = cv2.inRange(hsv,(0,0,0),(180,255,50))
    any_col = cv2.inRange(hsv,(0,0,0),(255,255,255))
    white_pix_cnt = cv2.countNonZero(mask_white)
    black_pix_cnt = cv2.countNonZero(mask_black)
    tot = cv2.countNonZero(any_col)
    col = max(tot-white_pix_cnt-black_pix_cnt,white_pix_cnt,black_pix_cnt)
    if col == white_pix_cnt:
        return 'White'
    elif col == black_pix_cnt:
        return 'Black'
    return 'Other'

In [9]:
image = cv2.imread('im6.jpg')
detect_shirt_colour(image,boxes[0])

'Black'

**Function to process Image**

In [53]:
def process_image(image_path):
    image = cv2.imread(image_path)
    boxes = detect_people(image)
    male_count,female_count = 0,0
    ages = []
    cnt = 0
    for box in boxes:
        cnt+=1
        x,y,w,h = box
        person_img = image[y:y+h,x:x+w]
        shirt_colour = detect_shirt_colour(image,box)

        face_img = person_img.copy()
        face_img = cv2.cvtColor(face_img,cv2.COLOR_BGR2RGB)
        face_img = transform(face_img).unsqueeze(0)
        face_img = face_img.permute(0,2,3,1).numpy()
        

        predictions = age_gender_model.predict(face_img)
        gender = 'Male' if predictions[0][0] <= 0.5 else 'Female'
        age = int(predictions[1][0])

        if shirt_colour == 'White':
            age = 23
        elif shirt_colour == 'Black' and len(boxes)>2:
            age = 'Child'


        if gender == "Male":
            male_count+=1
        else:
            female_count+=1
        ages.append(age)
        
        label = f'{gender}, Age: {age}'
        if cnt%2==0:
            cv2.rectangle(image,(x,y),((x+w),(y+h)),(0,255,0),1)
            cv2.putText(image,label,(x,y),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,150,0),1)
        else:
            cv2.rectangle(image,(x,y),((x+w),(y+h)),(0,255,0),1)
            cv2.putText(image,label,(x,y+h),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,150,0),1)
        
    
    return image, male_count,female_count,ages


### Main Function

**It takes the image path as arguement and saves a copy of processed image in our directory**

In [54]:
def people(image_path):
    processed_image,male_count,female_count,ages = process_image(image_path)
    print(f'No of Males: {male_count},No of Females: {female_count}')
    output_path = 'processed_'+image_path.split('/')[-1]
    cv2.imwrite(output_path,processed_image)
    print(ages)
    #cv2.imshow('Processed Image',processed_image)

In [55]:
image_path = 'mr1.jpeg'
people(image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step


C:\Users\hp\AppData\Local\Temp\ipykernel_11920\1747718727.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  age = int(predictions[1][0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
No of Males: 9,No of Females: 0
['Child', 54, 0, 0, 11, 8, 15, 0, 9]


In [90]:
image_path = 'grp_im1.jpeg'
people(image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step


C:\Users\hp\AppData\Local\Temp\ipykernel_9468\518462213.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  age = int(predictions[1][0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
No of Males: 5,No of Females: 3
[3, 23, 12, 23, 6, 7, 62, 0]


In [91]:
image_path = 'mr2.jpg'
people(image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


C:\Users\hp\AppData\Local\Temp\ipykernel_9468\518462213.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  age = int(predictions[1][0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
No of Males: 5,No of Females: 3
[0, 8, 23, 63, 0, 0, 45, 0]


In [92]:
image_path = 'mr3.jpeg'
people(image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


C:\Users\hp\AppData\Local\Temp\ipykernel_9468\518462213.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  age = int(predictions[1][0])


No of Males: 2,No of Females: 1
[0, 0, 47]


In [93]:
image_path = 'mr4.jpeg'
people(image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step


C:\Users\hp\AppData\Local\Temp\ipykernel_9468\518462213.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  age = int(predictions[1][0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
No of Males: 5,No of Females: 3
[2, 18, 10, 15, 21, 'Child', 3, 0]


In [85]:
image_path = 'mr5.jpeg'
people(image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


C:\Users\hp\AppData\Local\Temp\ipykernel_9468\3272911160.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  age = int(predictions[1][0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
No of Males: 5,No of Females: 0
[0, 10, 23, 41, 56]


In [86]:
image_path = 'mr6.jpeg'
people(image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


C:\Users\hp\AppData\Local\Temp\ipykernel_9468\3272911160.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  age = int(predictions[1][0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
No of Males: 6,No of Females: 1
[4, 2, 0, 7, 2, 2, 'Child']


In [69]:
image_path = 'mr7.jpeg'
people(image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step

C:\Users\hp\AppData\Local\Temp\ipykernel_9468\3506881900.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  age = int(predictions[1][0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
No of Males: 4,No of Females: 3
[23, 26, 5, 4, 4, 4, 11]


In [70]:
image_path = 'mr8.jpeg'
people(image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


C:\Users\hp\AppData\Local\Temp\ipykernel_9468\3506881900.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  age = int(predictions[1][0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
No of Males: 4,No of Females: 0
[61, 'Child', 0, 0]


In [71]:
image_path = 'mr9.jpeg'
people(image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


C:\Users\hp\AppData\Local\Temp\ipykernel_9468\3506881900.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  age = int(predictions[1][0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
No of Males: 3,No of Females: 3
[23, 4, 42, 3, 2, 0]


In [72]:
image_path = 'mr10.jpeg'
people(image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
No of Males: 1,No of Females: 0
[3]


C:\Users\hp\AppData\Local\Temp\ipykernel_9468\3506881900.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  age = int(predictions[1][0])


In [12]:
image_path = 'im6.jpg'
people(image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
No of Males: 1,No of Females: 0
[7]


C:\Users\hp\AppData\Local\Temp\ipykernel_11920\518462213.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  age = int(predictions[1][0])
